In [12]:
import requests
from bs4 import BeautifulSoup
import time
import numpy
import pandas as pd

## Scrape Event Page

In [2]:
## Store in the list
base_url = 'https://visitseattle.org/events/page/'
detail_links = [] 

for i in range(1, 42):
    url = base_url + str(i)
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')

    selector = 'div.search-result-preview > div > h3 > a'
    detail_eles = soup.select(selector)
    # detail_list.append([x['href'] for x in detail_eles])
    for ele in detail_eles:
        detail_links.append(ele['href'])

In [3]:
# save the detail_links into pickle
import pickle
with open('detail_links.pkl', 'wb') as f:
    pickle.dump(detail_links, f)
# read the pickle
with open('detail_links.pkl', 'rb') as f:
    detail_links = pickle.load(f)

In [4]:
## Print out the list
for link in detail_links:
    print(link)

https://visitseattle.org/events/john-cole-peter-de-lory/
https://visitseattle.org/events/the-puppet-masters-making-fools-of-us-all/
https://visitseattle.org/events/whim-whim-winter-24/
https://visitseattle.org/events/compania-nacional-de-danza/
https://visitseattle.org/events/metropolis/
https://visitseattle.org/events/andy-coe-band/
https://visitseattle.org/events/align/
https://visitseattle.org/events/billy-idol-and-duran-duran-tribute-night/
https://visitseattle.org/events/crack-sabbath/
https://visitseattle.org/events/harlem-globetrotters/
https://visitseattle.org/events/inked-up-conversation-with-john-ellis/
https://visitseattle.org/events/jukebox-quartet-arcobaleno-strings/
https://visitseattle.org/events/pike-place-market-local-appreciation-day/
https://visitseattle.org/events/seattle-baroque-orchestra-party-bohemienne/
https://visitseattle.org/events/seattle-thunderbirds-vs-everett-silvertips-2/
https://visitseattle.org/events/the-budos-band/
https://visitseattle.org/events/the

## Scrape Detail Page

In [72]:
event_list = []

for event in detail_links:
    res = requests.get(event)
    soup = BeautifulSoup(res.text, 'html.parser')
    name_selector = '.medium-6.columns.event-top > h1'
    date_selector = '.medium-6.columns.event-top > h4 > span:nth-child(1)'
    location_selector = '.medium-6.columns.event-top > h4 > span:nth-child(2)'
    type_selector = '.medium-6.columns.event-top > a:nth-child(3)'
    region_selector = '.medium-6.columns.event-top > a:nth-child(4)'

    name = soup.select(name_selector)
    date = soup.select(date_selector)
    location = soup.select(location_selector)
    Type = soup.select(type_selector)
    region = soup.select(region_selector)
    event_list.append({'Name': name[0].text, 'Date': date[0].text, 'Location': location[0].text.strip(), 'Type': Type[0].text, 'Region': region[0].text})
    time.sleep(1)

df = pd.DataFrame(event_list)
df.to_csv('event_data.csv', index=False)

    

In [73]:
# save the event_list into pickle
with open('event_list.pkl', 'wb') as f:
    pickle.dump(event_list, f)
# read the pickle
with open('event_list.pkl', 'rb') as f:
    event_list = pickle.load(f)

## Weather API

In [89]:
Location_list = [event['Location'] for event in event_list]

In [95]:
map_url = "https://nominatim.openstreetmap.org/search"
weather_list = []

for i in range(len(Location_list)):
    print(Location_list[i])
    query_params = {
        "q": Location_list[i],
        "format": "jsonv2"
    }
    res = requests.get(map_url, params=query_params)
    map_dict = res.json()
    if map_dict == []:
        weather_list.append({'Location': Location_list[i], 'Day': 'No data', 'Temperature': 'No data', 'ShortForecast': 'No data'})
        continue
    latitude = map_dict[0]['lat']
    longitude = map_dict[0]['lon']
    
    weather_url = "https://api.weather.gov/points/{},{}".format(latitude, longitude)
    res = requests.get(weather_url)
    point_dict = res.json()
    print(weather_url)

    if 'properties' not in point_dict:
        print('No weather data')
        weather_list.append({'Location': Location_list[i], 'Day': 'No data', 'Temperature': 'No data', 'ShortForecast': 'No data'})
        continue
    else:
        forcast_url = point_dict['properties']['forecast']
        res = requests.get(forcast_url)
        weather_dict = res.json()

    if 'properties' not in weather_dict:
        print(forcast_url)
        weather_list.append({'Location': Location_list[i], 'Day': 'No data', 'Temperature': 'No data', 'ShortForecast': 'No data'})
        continue
    
    weather = weather_dict['properties']['periods']
    for x in weather:
        if x['isDaytime'] == True:
            weather_list.append({'Location': Location_list[i], 'Day': x['name'], 'Temperature': x['temperature'], 'ShortForecast': x['shortForecast']})


df = pd.DataFrame(weather_list)
df.to_csv('weather_data.csv', index=False)
weather_list

Harris Harvey Gallery
Gallery 110
https://api.weather.gov/points/47.6012826,-122.330069
Cornish Playhouse at Seattle Center
Meany Center for the Performing Arts
Benaroya Hall
https://api.weather.gov/points/47.60811665,-122.33697040702705
Nectar Lounge
https://api.weather.gov/points/47.6523876,-122.3538541520202
High Dive
https://api.weather.gov/points/41.7111625,-93.6318959
Sunset Tavern
https://api.weather.gov/points/36.0254577,-114.9637699
Sea Monster Lounge
https://api.weather.gov/points/47.6615049,-122.332319
Climate Pledge Arena
https://api.weather.gov/points/47.6219014,-122.353990491486
Bainbridge Island Museum of Art
https://api.weather.gov/points/47.6251322,-122.51453295865404
https://api.weather.gov/gridpoints/SEW/119,70/forecast
The Royal Room
https://api.weather.gov/points/32.2275723,-110.9685178
Pike Place Market
https://api.weather.gov/points/47.60939675,-122.34141017673986
Town Hall Seattle
https://api.weather.gov/points/47.6090191,-122.32993774092222
accesso ShoWare Cent

[{'Location': 'Harris Harvey Gallery',
  'Day': 'No data',
  'Temperature': 'No data',
  'ShortForecast': 'No data'},
 {'Location': 'Gallery 110',
  'Day': 'This Afternoon',
  'Temperature': 54,
  'ShortForecast': 'Rain'},
 {'Location': 'Gallery 110',
  'Day': 'Sunday',
  'Temperature': 59,
  'ShortForecast': 'Light Rain Likely'},
 {'Location': 'Gallery 110',
  'Day': 'Monday',
  'Temperature': 63,
  'ShortForecast': 'Light Rain Likely'},
 {'Location': 'Gallery 110',
  'Day': 'Tuesday',
  'Temperature': 57,
  'ShortForecast': 'Light Rain Likely'},
 {'Location': 'Gallery 110',
  'Day': 'Wednesday',
  'Temperature': 53,
  'ShortForecast': 'Light Rain'},
 {'Location': 'Gallery 110',
  'Day': 'Thursday',
  'Temperature': 50,
  'ShortForecast': 'Light Rain Likely'},
 {'Location': 'Gallery 110',
  'Day': 'Friday',
  'Temperature': 45,
  'ShortForecast': 'Chance Light Rain'},
 {'Location': 'Cornish Playhouse at Seattle Center',
  'Day': 'No data',
  'Temperature': 'No data',
  'ShortForecast'